In [1]:
import os
import librosa
import numpy as np
from PIL import Image

def extract_mel_spectrogram(file_path, n_mels=128, hop_length=64, fixed_length=160, sr=22050, duration=0.5):
    """
    從音訊文件中提取梅爾頻譜圖。
    
    參數:
    - file_path (str): 音訊文件路徑
    - n_mels (int): 梅爾頻帶數量
    - hop_length (int): 每次窗口移動的樣本數
    - fixed_length (int): 固定的時間步數（橫向維度）
    - sr (int): 採樣率
    - duration (float): 音訊片段時長（秒）
    
    返回:
    - mel_spec_db (np.ndarray): 處理後的梅爾頻譜圖
    """
    try:
        # 加載音訊文件
        y, sr = librosa.load(file_path, sr=sr, duration=duration)
        
        # 確保音訊長度固定，pad 或 truncate
        y = librosa.util.fix_length(y, size=int(sr * duration))
        
        # 提取梅爾頻譜圖
        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, hop_length=hop_length)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
        
        # 填充或截斷時間步數
        if mel_spec_db.shape[1] < fixed_length:
            pad_width = fixed_length - mel_spec_db.shape[1]
            mel_spec_db = np.pad(mel_spec_db, pad_width=((0,0), (0, pad_width)), mode='constant')
        else:
            mel_spec_db = mel_spec_db[:, :fixed_length]
        
        # 重塑或插值至 (固定高度 x 固定寬度)
        mel_image = Image.fromarray(mel_spec_db)
        mel_image = mel_image.resize((128, fixed_length))  # (width, height)
        mel_spec_db = np.array(mel_image)
        
        return mel_spec_db.astype(np.float32)
    
    except Exception as e:
        print(f"處理文件 {file_path} 時出現錯誤: {e}")
        return None


In [2]:
def create_dataset(swallow_dir, non_dir, n_mels=128, hop_length=64, fixed_length=160, sr=22050, duration=0.5):
    """
    建立資料集。
    
    參數:
    - swallow_dir (str): 吞嚥聲音檔案資料夾路徑
    - non_dir (str): 非吞嚥聲音檔案資料夾路徑
    - 其他參數同上
    
    返回:
    - data (np.ndarray): 梅爾頻譜圖數據
    - labels (np.ndarray): 標籤數據
    """
    data = []
    labels = []
    
    # 加載吞嚥聲音檔案
    for file in os.listdir(swallow_dir):
        if file.endswith('.wav'):
            file_path = os.path.join(swallow_dir, file)
            mel = extract_mel_spectrogram(file_path, n_mels=n_mels, hop_length=hop_length, 
                                         fixed_length=fixed_length, sr=sr, duration=duration)
            if mel is not None:
                data.append(mel)
                labels.append(1)
    
    # 加載非吞嚥聲音檔案
    for file in os.listdir(non_dir):
        if file.endswith('.wav'):
            file_path = os.path.join(non_dir, file)
            mel = extract_mel_spectrogram(file_path, n_mels=n_mels, hop_length=hop_length, 
                                         fixed_length=fixed_length, sr=sr, duration=duration)
            if mel is not None:
                data.append(mel)
                labels.append(0)
    
    return np.array(data), np.array(labels)

def add_background_noise(y, noise_factor=0.005):
    """
    向音訊添加背景噪音。
    
    參數:
    - y (np.ndarray): 音訊數據
    - noise_factor (float): 噪音強度因子
    
    返回:
    - y_noisy (np.ndarray): 添加噪音後的音訊數據
    """
    noise = np.random.randn(len(y))
    y_noisy = y + noise_factor * noise
    y_noisy = y_noisy.astype(type(y[0]))
    return y_noisy

def adjust_volume(y, factor=0.8):
    """
    調整音訊的音量。
    
    參數:
    - y (np.ndarray): 音訊數據
    - factor (float): 音量調整因子
    
    返回:
    - y_adjusted (np.ndarray): 調整音量後的音訊數據
    """
    return y * factor


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 定義資料夾路徑
swallow_dir = 'sound_split_data/swallow'  # 替換為實際路徑
non_dir = 'sound_split_data/non'          # 替換為實際路徑

# 建立資料集
X, y = create_dataset(swallow_dir, non_dir, sr=22050)
print(f"資料集大小: {X.shape}, 標籤大小: {y.shape}")  # 應為 (num_samples, 160, 128)

# 移除可能的空值樣本
valid_indices = [i for i in range(len(X)) if X[i] is not None]
X = X[valid_indices]
y = y[valid_indices]

# 標準化
scaler = StandardScaler()
X_reshaped = X.reshape(X.shape[0], -1)  # 將每個頻譜圖展平成一維 (num_samples, 160*128=20480)
X_scaled = scaler.fit_transform(X_reshaped)  # (num_samples, 20480)

# 重塑為 (num_samples, 160, 128, 1)
X_scaled = X_scaled.reshape(X.shape[0], 160, 128, 1)

# 複製通道以符合模型要求 (num_samples, 160, 128, 3)
X_scaled = np.repeat(X_scaled, 3, axis=-1)  # 現在形狀為 (num_samples, 160, 128, 3)

# 確認數據形狀
print(f"重塑後資料集大小: {X_scaled.shape}, 標籤大小: {y.shape}")  # (num_samples, 160, 128, 3), (num_samples,)

# 拆分訓練集和驗證集
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)
print(f"訓練集大小: {X_train.shape}, 驗證集大小: {X_val.shape}")  # (num_train, 160, 128, 3), (num_val,)


In [ ]:
from tensorflow.keras import layers, models

def build_simple_cnn_lstm(input_shape):
    inputs = layers.Input(shape=input_shape)
    
    # 簡單的 CNN 模塊
    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2,2))(x)  # (80, 64, 32)
    x = layers.Dropout(0.25)(x)
    
    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2,2))(x)  # (40, 32, 64)
    x = layers.Dropout(0.25)(x)
    
    x = layers.Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2,2))(x)  # (20, 16, 128)
    x = layers.Dropout(0.25)(x)
    # Flatten 和 Dense
    x = layers.Flatten()(x)  # (20*16*128, )
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.25)(x)
    
    # 為 LSTM 準備序列格式
    x = layers.Reshape((1, 128))(x)  # (1, 256)
    x = layers.LSTM(64, return_sequences=False, recurrent_dropout=0.2)(x)
    
    # 輸出層
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = models.Model(inputs, outputs)
    return model

input_shape = (160, 128, 3)  # (fixed_length, n_mels, channels)
model = build_simple_cnn_lstm(input_shape)

# 編譯模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
# 訓練模型
history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    ]
)

# 繪製訓練與驗證的準確率
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train_accuracy')
plt.plot(history.history['val_accuracy'], label='Val_accuracy')
plt.legend()
plt.title('accuracy')

# 繪製訓練與驗證的損失
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train_loss')
plt.plot(history.history['val_loss'], label='Val_loss')
plt.legend()
plt.title('loss')
plt.show()



In [5]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc

# 在驗證集上進行預測
y_pred_prob = model.predict(X_val)
y_pred = (y_pred_prob > 0.5).astype(int)
# 混淆矩陣
cm = confusion_matrix(y_val, y_pred)
print("混淆矩陣:")
print(cm)
# 分類報告
cr = classification_report(y_val, y_pred, target_names=['Non-Swallow', 'Swallow'])
print("分類報告:")
print(cr)

fpr, tpr, thresholds = roc_curve(y_val, y_pred_prob)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('FP')
plt.ylabel('TP')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()



NameError: name 'model' is not defined

In [ ]:
def count_swallowing(file_path, model, scaler, n_mels=128, hop_length=64, fixed_length=160, sr=22050):
    """
    計算單一音訊文件中的吞嚥聲數量。
    
    參數:
    - file_path (str): 音訊文件路徑
    - model (tf.keras.Model): 訓練好的模型
    - scaler (StandardScaler): 訓練階段使用的標準化器
    - n_mels (int): 梅爾頻帶數量
    - hop_length (int): 每次窗口移動的樣本數
    - fixed_length (int): 固定的時間步數（橫向維度）
    - sr (int): 採樣率
    
    返回:
    - count (int): 檢測到的吞嚥聲數量
    """
    y, sr = librosa.load(file_path, duration=30, sr=sr)
    chunk_length = int(sr * 0.5)  # 0.5秒
    count = 0
    for i in range(0, len(y), chunk_length):
        chunk = y[i:i+chunk_length]
        if len(chunk) < chunk_length:
            # 填充
            chunk = librosa.util.fix_length(chunk, size=chunk_length)
        mel_spec = librosa.feature.melspectrogram(y=chunk, sr=sr, n_mels=n_mels, hop_length=hop_length)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
        # 填充或截斷
        if mel_spec_db.shape[1] < fixed_length:
            pad_width = fixed_length - mel_spec_db.shape[1]
            mel_spec_db = np.pad(mel_spec_db, pad_width=((0,0), (0, pad_width)), mode='constant')
        else:
            mel_spec_db = mel_spec_db[:, :fixed_length]
        # 重塑或插值至160x128
        mel_image = Image.fromarray(mel_spec_db)
        mel_image = mel_image.resize((128, fixed_length))  # (width, height)
        mel_spec_db = np.array(mel_image)
        # 標準化
        X_scaled = scaler.transform(mel_spec_db.reshape(1, -1)).reshape(mel_spec_db.shape)
        # 調整形狀
        X_scaled = X_scaled.reshape(1, fixed_length, n_mels, 1)
        # 複製通道以符合模型要求
        X_scaled = np.repeat(X_scaled, 3, axis=-1)  # 現在形狀為 (1, 160, 128, 3)
        # 預測
        prediction = model.predict(X_scaled)
        if prediction > 0.5:
            print(f"swallow conf:{prediction}")
            count += 1
    return count

# 範例用法
file_path = '音檔\測試RSST.wav'  # 替換為實際路徑
num_swallow = count_swallowing(file_path, model, scaler)
print(f"吞嚥聲數量: {num_swallow}")
